# Calcul matriciel et pièges en numpy

Un des principaux défauts de ```numpy``` (par rapport à des environnement comme Matlab par exemple) est la **différentiation entre les vecteurs et les matrices**... Cet aspect va poser de nombreux problèmes.

Le second défaut réside dans le **dispatch dynamique**: python essaie de faire marcher les calculs, même quand les dimensions des matrices sont incompatibles... Ca fait gagner un peu de temps parfois... Et ça crée des bugs insurmontables d'autres fois.

In [1]:
import numpy as np

# Produit matriciel

Il ne faut pas confondre le produit terme à terme, entre matrices de mêmes dimensions et le produit matriciel qui est un ensemble de produits scalaires. 
Dans le produit matriciel, le nombre de colonne de la première matrice doit correspondre au nombre de ligne de la seconde/


<img src="./ressources/matrix_mult.png">

$$ {\color {red}c_{{12}}}=\sum _{{r=1}}^{2}a_{{1r}}b_{{r2}}=a_{{11}}b_{{12}}+a_{{12}}b_{{22}}$$

$$ {\color {blue}c_{{33}}}=\sum _{{r=1}}^{2}a_{{3r}}b_{{r3}}=a_{{31}}b_{{13}}+a_{{32}}b_{{23}}$$

In [2]:
A = np.array([[0., 1], [2, 3], [4, 5], [6, 7]])
B = np.array([[5., 4, 3], [2, 1, 0]])
print("A=",A,"\n B=",B)

# calcul de produit matriciel, par différents moyens
C  = A@B # le plus clair d'après moi
C2 = A.dot(B)

print(C)

A= [[0. 1.]
 [2. 3.]
 [4. 5.]
 [6. 7.]] 
 B= [[5. 4. 3.]
 [2. 1. 0.]]
[[ 2.  1.  0.]
 [16. 11.  6.]
 [30. 21. 12.]
 [44. 31. 18.]]


In [3]:
# test sur des matrices incompatibles (matriciel)

A = np.array([[0., 1], [2, 3], [4, 5], [6, 7]])
M = np.array([[0., 1], [2, 3], [4, 5]])

print(A@M)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 2)

In [ ]:
# et le produit terme à terme, uniquement entre matrices de même dimension

A = np.array([[0., 1], [2, 3], [4, 5], [6, 7]])
A2 = np.array([[1., 2], [2, 1], [1, 2], [2, 1]])

print(A*A2)

In [4]:
# test sur des données incompatibles (terme à terme)

C3 = A*B # tournure historiquement très ambigue : 
# heureusement, ne marche plus sur les nouvelles versions de python
# le * est maintenant réservé aux mutliplications terme à terme

ValueError: operands could not be broadcast together with shapes (4,2) (2,3) 

## Types de données

In [5]:
# Les commandes ci-dessous créent des vecteurs

v1=np.random.rand(10)
v2=np.array([1, 4, 18])
v3=np.ones(12)

print(v1.shape)

(10,)


In [6]:
# Les commandes ci-dessous créent des matrices

m1=np.random.rand(10,2)
m2=np.array([[1, 4, 18],[2, 4, 6]])
m3=np.ones((12,2))

print(m1.shape)

(10, 2)


In [7]:
# Cas limite

v2 = np.array([1, 4, 18])
# et 
m2 = np.array([[1, 4, 18]])

print("v2 et m2 ne sont pas du même type: ", v2.shape, m2.shape)

v2 et m2 ne sont pas du même type:  (3,) (1, 3)


In [8]:
# extraire une ligne ou une colonne => générer un vecteur (et pas une matrice)
# => nous sommes obligé de jongler avec les types de données

m1 = np.random.rand(10,3)
v4 = m1[:,1]   # extraction d'une colonne => vecteur
m4 = m1[:,1:3] # extraction de deux colonnes => matrice
x1 = m1[:,1:2] # extraction d'une seule colonnes, mais en syntaxe matricielle => ???

print(v4.shape, m4.shape)
print(x1.shape) # => matrice !!

(10,) (10, 2)
(10, 1)


### Pourquoi ces différences de types posent problème?

In [9]:
A     = np.array([[0., 1], [2, 3], [4, 5], [6, 7]])
B_col = np.array([[1.], [2]]) # matrice (=en forme de vecteur colonne)
B_li  = np.array([[1., 2]])   # matrice (=en forme de vecteur ligne)
B_vec = np.array([1., 2])     # vecteur

# print(A@B_li) # KO pour les dimensions => raisonnable, mais attention aux versions de python
print(A@B_col)
print(A@B_vec) # les résultats n'ont pas les mêmes dimensions
print(A.dot(B_col))
print(A.dot(B_vec)) # les résultats n'ont pas les mêmes dimensions

print("\n")

# print(A*B_col) # Erreur => c'est logique
print(A*B_li)    # Catastrophe => ca ne fait pas d'erreur
print(A*B_vec)   # Catastrophe => ca ne fait pas d'erreur

[[ 2.]
 [ 8.]
 [14.]
 [20.]]
[ 2.  8. 14. 20.]
[[ 2.]
 [ 8.]
 [14.]
 [20.]]
[ 2.  8. 14. 20.]


[[ 0.  2.]
 [ 2.  6.]
 [ 4. 10.]
 [ 6. 14.]]
[[ 0.  2.]
 [ 2.  6.]
 [ 4. 10.]
 [ 6. 14.]]


In [10]:
B_col = np.array([[1.], [2]]) # matrice (=en forme de vecteur colonne)
B_li  = np.array([[1., 2]])   # matrice (=en forme de vecteur ligne)
B_vec = np.array([1., 2])     #

print(B_li@B_col) # le calcul est propre... Et renvoie une matrice
# print(B_li@B_li)  # KO
print(B_vec@B_vec) # produit scalaire
print(B_vec@B_col) # renvoie un vecteur


[[5.]]
5.0
[5.]


In [11]:
# tentons dans l'autre sens
# 
print(B_col @ B_li) # renvoie une matrice (OK)
# print(B_col @ B_vec) # KO : alors qu'on  voudrait intuitivement que ça marche
print(B_vec * B_li) # OK
print(B_vec * B_col) # => WAOUH , carrement n'importe quoi !!

[[1. 2.]
 [2. 4.]]
[[1. 4.]]
[[1. 2.]
 [2. 4.]]


## Dispatch dynamique

Beaucoup de comportements étranges sont liés à cette fonctionnalité. Partons d'un exemple simple et étudions les différentes solutions pratiques:

    # Soit la matrice A:
    A = np.array([[0., 1], [2, 3], [4, 5], [6, 7]])
    # Je souhaite multiplier chaque ligne par le vecteur [1,2]

In [12]:
# Soit la matrice A:
A = np.array([[0., 1], [2, 3], [4, 5], [6, 7]])

# Je souhaite multiplier chaque ligne par le vecteur [1,2]


In [13]:
# solution 1: developpeur standard avec des boucles

B = A.copy()
for ligne in B:
    ligne *= [1,2]
print(B)

[[ 0.  2.]
 [ 2.  6.]
 [ 4. 10.]
 [ 6. 14.]]


In [14]:
# solution 2: raisonnement matriciel, je veux juste multiplier la seconde colonne
B = A.copy()
B[:,1] *= 2
print(B)

[[ 0.  2.]
 [ 2.  6.]
 [ 4. 10.]
 [ 6. 14.]]


In [15]:
# solution 3: je crée une matrice pour ensuite faire une multiplication terme à terme

M = np.ones((4,2))
M[:,1] *= 2
B = A*M
print(B)

[[ 0.  2.]
 [ 2.  6.]
 [ 4. 10.]
 [ 6. 14.]]


In [16]:
# solution 4: dispatch dynamique
# ça ne devrait pas marcher... MAIS
#   1. python détecte que le nb de colonne est compatible
#   2. python applique l'opération sur chaque ligne automatiquement
#   => pratique... Mais risqué: il faut connaitre ce truc pour détecter les bugs associés

B = A * [1,2]
print(B)

[[ 0.  2.]
 [ 2.  6.]
 [ 4. 10.]
 [ 6. 14.]]


In [17]:
# pour faire la même chose sur les colonnes
# à multiplier par [1, 2, 3, 4]

B = A * [[1],[2],[3],[4]] # il faut présenter un vecteur colonne 
print(B)

[[ 0.  1.]
 [ 4.  6.]
 [12. 15.]
 [24. 28.]]


## <span style="color:red"> EXO produit divers</span>

Soit une matrice de dimension $(n,d)$ remplie de 1. Chercher la formule qui permet de passer à une matrice $[1,2,...,d]$ répété respectivement sur les lignes ou les colonnes:

$ M = \begin{pmatrix}
1 & 1 & \ldots &1 \\
1 & 1 & \ldots & 1 \\
\vdots &  \vdots & \ddots & \vdots \\
1 & 1 & \ldots & 1 \\
\end{pmatrix} 
\Rightarrow 
\begin{pmatrix}
1 & 2 & \ldots &d \\
1 & 2 & \ldots & d \\
\vdots &  \vdots & \ddots & \vdots \\
1 & 2 & \ldots & d \\
\end{pmatrix} \text{ ou }
\begin{pmatrix}
1 & 1 & \ldots &1 \\
2 & 2 & \ldots & 2 \\
\vdots &  \vdots & \ddots & \vdots \\
n & n & \ldots & n \\
\end{pmatrix}$

In [38]:
M=np.ones((8,8))
"""N=np.array(([1,0,0,0,0,0,0,0],
           [0,2,0,0,0,0,0,0],
           [0,0,3,0,0,0,0,0],
           [0,0,0,4,0,0,0,0],
           [0,0,0,0,5,0,0,0],
           [0,0,0,0,0,6,0,0],
           [0,0,0,0,0,0,7,0],
           [0,0,0,0,0,0,0,8]))"""
N=np.array([1,2,3,4,5,6,7,8])
print(M*N)
print((N*M).T)
#faut que pour N la somme des colonnes soit égal à l'indice +1

[[1. 2. 3. 4. 5. 6. 7. 8.]
 [1. 2. 3. 4. 5. 6. 7. 8.]
 [1. 2. 3. 4. 5. 6. 7. 8.]
 [1. 2. 3. 4. 5. 6. 7. 8.]
 [1. 2. 3. 4. 5. 6. 7. 8.]
 [1. 2. 3. 4. 5. 6. 7. 8.]
 [1. 2. 3. 4. 5. 6. 7. 8.]
 [1. 2. 3. 4. 5. 6. 7. 8.]]
[[1. 1. 1. 1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2. 2. 2. 2.]
 [3. 3. 3. 3. 3. 3. 3. 3.]
 [4. 4. 4. 4. 4. 4. 4. 4.]
 [5. 5. 5. 5. 5. 5. 5. 5.]
 [6. 6. 6. 6. 6. 6. 6. 6.]
 [7. 7. 7. 7. 7. 7. 7. 7.]
 [8. 8. 8. 8. 8. 8. 8. 8.]]


## <span style="color:red"> EXO Evaluation d'une fonction linéaire</span>

Soit une matrice $X$ de dimension $(n,d)$ stockant $n$ échantillon $\mathbf x_i \in \mathbb R^d$ et un vecteur de paramètres $w \in \mathbb R^d$, 

$ X = \begin{pmatrix}
x_{11} & x_{12} & \ldots & x_{1d} \\
x_{21} & x_{22} & \ldots & x_{2d} \\
\vdots &  \vdots & \ddots & \vdots \\
x_{n1} & x_{n2} & \ldots & x_{nd} \\
\end{pmatrix} 
\qquad
W = \begin{pmatrix}
w_{1} & w_{2} & \ldots & w_{d} 
\end{pmatrix} 
$

Calculer l'ensemble : $\forall i, f(\mathbf x_i) = \sum_{j=1}^d x_{ij} w_j $
Cet ensemble sera stocké dans un vecteur.

In [33]:
n = 100
d = 10
X = np.random.rand(n,d)
#X=np.ones((n,d))
W = np.arange(d)
C=X*W
D=np.sum(C,axis=1)
print(D)


[14.60143122 22.85353289 19.49275029 19.73970254 22.00133607 29.25675736
 29.69988132 23.09093416 16.72303769 16.15586465 17.82568914 30.68130028
 23.46713442 15.97487149 25.62426922 24.04411936 21.09414281 27.87101329
 26.66611964 23.90079085 21.67014433 19.93977299 25.0924835  16.53421908
 14.42997208 17.79629299 18.09393995 28.62385528 27.77415429 24.59433516
 23.24096438 18.10258603 18.30099402  9.81035747 23.53796387 19.01597981
 17.97980313 19.62620675 18.8439676  19.69269579 26.03418516 24.55468252
 22.35194956 31.90676455 22.09335445 21.73346314 23.17173614 25.12312508
 29.09369167 25.10315344 15.64459925 22.94159548 18.63339177 25.4621978
 26.4122006  24.3792796  13.5632725  29.59140593 36.23735934 22.0612412
 19.98513327 27.43440681 25.47838019 28.27064475 21.36984927 30.69548087
 19.86880697 19.12426665 20.22986317 22.96823027 18.46497774 19.96786224
 21.16396438 25.69534381 25.8130571  15.28945924 28.12781328 25.46439373
 22.39577302 22.56811894 17.78014239 26.56819099 26.7